In [ ]:

!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l
    

## 安装kaggle、凭证（$\color{#FF0000}{记得转入kaggle.json}$）

In [35]:
!pip install matplotlib
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json  /root/.kaggle
!chmod 600 ~/.kaggle/kaggle.json


!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions download -c classify-leaves 

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
classify-leaves.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
from torchvision import datasets, transforms
import os
import zipfile
import shutil
import random
import matplotlib as plt

##  解压数据

In [4]:


def un_zip(file_name):
    """unzip zip file"""
    base_dir = os.path.dirname(file_name)
    zip_file = zipfile.ZipFile(file_name, 'r')
    os.makedirs(os.path.join(*file_name.split('.')[:-1])   ,exist_ok=True )
    zip_file.extractall(os.path.join(*file_name.split('.')[:-1]) )

un_zip('classify-leaves.zip')
len(os.listdir('classify-leaves'))

4

## 数据集处理01(训练集csv-->将数据放到指定文件夹)

In [5]:
#数据集 处理
def make_dataset(fpath, tpath = 'data'):
    
    
    data = pd.read_csv(fpath) # csv
    os.makedirs(os.path.join(tpath), exist_ok=True)
    for i in range(data.shape[0]):
        if data.shape[1] == 2: #训练集
            fname = os.path.join(tpath, 'trainset')# 训练集目标位置
            os.makedirs(fname, exist_ok=True)
            os.makedirs(os.path.join(fname, data.iloc[i,1]), exist_ok=True)
            shutil.copy(os.path.join(*fpath.split('/')[:-1], data.iloc[i, 0]), os.path.join(fname, data.iloc[i,1]))
            
        elif data.shape[1] == 1: #测试集
            fname = os.path.join(tpath, 'testset')
            os.makedirs(fname, exist_ok=True)
            shutil.copy(os.path.join( *fpath.split('/')[:-1], data.iloc[i, 0]), fname)
        else:
            assert 'make dataset error!'

make_dataset(os.path.join('classify-leaves', 'train.csv'), 'data')
make_dataset(os.path.join('classify-leaves', 'test.csv'), 'data')

##  数据集处理02(训练集 -->按比例随机将数据放入文件夹)

In [6]:

def make_testset(fpath, tpath, scale=.0): 
    '''func:将完整训练数据随机按scale 比例分成 train 和 test 文件夹
        parmarater:
        fpath:完好的训练数据文件夹， 
        tpath:目标文件夹'''

    os.makedirs(os.path.join(tpath), exist_ok=True)
    os.makedirs(os.path.join(tpath, 'train'), exist_ok=True)
    os.makedirs(os.path.join(tpath, 'test'), exist_ok=True)
    class_name = os.listdir(fpath)
#     num_scale = len(list_name)  #总共类别
    for class_leaves in class_name:
        imgname = os.listdir(os.path.join(fpath, class_leaves))  #每一类数据的文件夹
        testdir = os.path.join(tpath, 'test', class_leaves)
        traindir = os.path.join(tpath, 'train', class_leaves)
        
        os.makedirs(testdir, exist_ok=True)
        os.makedirs(traindir, exist_ok=True)
        for i in range(int(len(imgname) * scale)):
            j = random.randint(0, len(imgname) -1)
            
            shutil.copy(os.path.join(fpath, class_leaves, imgname[j]), testdir)
            del imgname[j]
        for img in imgname:
            shutil.copy(os.path.join(fpath, class_leaves, img), traindir)
 
        
make_testset(os.path.join('data', 'trainset'),'train', scale=0.3)

## resnet-11 模型

In [4]:

class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,
                use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, 
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, 
                               kernel_size=3, padding=1)
        
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, 
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [5]:
b1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,),
    nn.BatchNorm2d(64),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding =1))

In [6]:
def resnet_block(input_channels, num_channels, num_residuals, 
                first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, 
                               use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [7]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [8]:
net =nn.Sequential(b1, b2, b3, b4, b5, 
                  nn.AdaptiveAvgPool2d((1,1)),
                  nn.Flatten(), nn.Linear(512, 176))

In [9]:
X = torch.rand(size=(1, 3, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 176])


In [25]:
def train_ch6(net, trian_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('trianing on', device)
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1,num_epochs],
                           legend=['train loss', 'trian acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i+1) / num_batches, (train_l, train_acc, None))
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))                        
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}',
         f'test_acc {test_acc:.3f}')
    print(f'{metric[2] *num_epochs / timer.sum():.1f} examples/sec',
         f'on {str(device)}')
            

In [20]:
#图片处理流水线 resize 224, 转为向量 归一化
transform = transforms.Compose([transforms.Resize(224), 
                 transforms.ToTensor(), 
                transforms.RandomHorizontalFlip(),  # 随即反转
                # transforms.RandomSizedCrop(224),
                transforms.RandomCrop(96),
                # transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
                transforms.Normalize((0, 0, 0), (1, 1, 1))])
train_dataset = datasets.ImageFolder(os.path.join('train', 'train'), transform=transform)
test_dataset  = datasets.ImageFolder(os.path.join('train', 'test'), transform=transform)

train_iter = torch.utils.data.DataLoader(train_dataset,batch_size=32, shuffle=True, num_workers = 2)
test_iter = torch.utils.data.DataLoader(test_dataset,batch_size=32, shuffle=True, num_workers = 2)

images = next(iter(test_iter))

In [26]:

# def imshow(img):
#     # img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
# imshow(test_iter)

In [ ]:
lr, num_epochs, batch_size = 5, 10, 32


train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())



trianing on cuda:0
